In [47]:
import cv2 as cv
import matplotlib.pyplot as plt
import os
import pandas as pd 

In [2]:
class Crop_Mask(object):
    def __init__(self, mask_path, csv_files):
        self.mask_path = mask_path
        self.csv_files = csv_files
        
    def sort_list(self):
        png_list = self.get_png_list()
        num_list = []
        for idx, l in enumerate(png_list):
            num = l.split('_')[-1].split('.')[0]
            num_list.append(num)
        ziped_list = zip(map(int, num_list), png_list)
        sorted_list = sorted(ziped_list)
        sorted_png = []
        for _, ls in sorted_list:
            sorted_png.append(ls)
        print(sorted_png)
        return sorted_png
    
    def get_png_list(self):
        # arg: mask_path，为mask的路径（有多张）
        # arg: csv_files, 提供名称，用于寻找对应的mask文件
        # return: 获取相应的mask文件路径
        main_name = self.csv_files.split('\\')[-1].split('.')[0]
        print(main_name)
        mask_list = os.listdir(self.mask_path)
        self.main_name = main_name
        #print(mask_list)
        mask_file_path = []
        png_file_path_fpn = []
        png_file_path_v2 = []
        for png in mask_list:
            mask_file_path.append(os.path.join(self.mask_path , png))
        for mask in mask_file_path:
            main_v2 = mask.split('\\')[-1].split('v')[0] + 'v23'
            main_fpn = mask.split('\\')[-1].split('f')[0] + 'fpn'
            # print(main[0:8])
            if mask.split('\\')[-1].split('_')[-2] == 'fpn':
                if main_fpn == main_name:
                    png_file_path_fpn.append(mask)  
            else:
                if main_v2 == main_name:
                    png_file_path_v2.append(mask)
                    
        if len(png_file_path_fpn) == 0:
            assert len(png_file_path_v2) != 0, '出现错误...'
            return png_file_path_v2
        else:
            return png_file_path_fpn # png的文件路径
        
        
    def read_png_csv(self, show_img = False):
        # 功能：读取相应的png以及csv文件
        png_list = self.sort_list()
        print(png_list)
        assert type(png_list) == 'list' or 'tuple', 'png_list must be iterable'
        png_numpy_list = []
        for png in png_list:
            png_numpy_list.append(cv.imread(png))
        png_list_rgb = [cv.cvtColor(png, cv.COLOR_BGR2RGB) for png in png_numpy_list]
        for i in png_list_rgb:
            assert i.shape == (768, 1024, 3), 'the image shape is {0}'.format(i.shape)
        df_i = pd.read_csv(self.csv_files)
        if show_img == True:
            for i in range(len(png_numpy_list)):
                df_m = df_i.loc[i,:]
                x1, y1, x2, y2 = df_m['x1'], df_m['y1'], df_m['x2'], df_m['y2']
                x1, y1, x2, y2 = map(int, (x1, y1, x2, y2))
                mask_i = png_list_rgb[i]
                cv.rectangle(mask_i, (x1 ,y1), (x2 ,y2 ), (255, 0, 0), thickness = 2)
                cv.imshow('Window Name', mask_i)
                cv.waitKey(0)
            cv.destroyAllWindows()
        return df_i, png_list_rgb
    
    def Crop_Masks(self, save_path=None, is_show = False, is_return=False):
        df, png_list_rgb = self.read_png_csv()
        png_path_list = self.get_png_list()
        list_name = [name.split('\\')[-1].split('.')[0] for name in png_path_list]
        crop_img = []
        for i in range(len(png_list_rgb)):
            df_m = df.loc[i,:]
            x1, y1, x2, y2 = df_m['x1'], df_m['y1'], df_m['x2'], df_m['y2']
            x1, y1, x2, y2 = map(int, (x1, y1, x2, y2))
            mask_i = png_list_rgb[i]
            cropped_image = mask_i[y1 : y2, x1  : x2]  # 使用切片操作进行图像剪裁
            crop_img.append(cropped_image)
            if is_show == True:
                plt.imshow(cropped_image, interpolation='nearest')
        if save_path:
            if os.path.exists(save_path) is False:
                os.makedirs(save_path)
            for i, mask in enumerate(crop_img):
                file = list_name[i]
                png_name = file + '_' + 'mask' + '.jpg'
                png_name_path = os.path.join(save_path, png_name)
                cv.imwrite(png_name_path, mask)
        if is_return:
            return crop_img 